# Exercice semaine 4: Correction tests multiples et biomarqueurs

## Chargement des libraries

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
from scipy import stats

%matplotlib inline
import matplotlib.pyplot as plt
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.compare import compare_survival

## Partie 1 : Introduction aux tests multiples

1) Faîtes 1000 t-tests entre des distributions normales centrées réduites de taille 30 générées aléatoirement. Afficher les deux dernières distributions générées.

In [ ]:
resRandom = list()
for g in range(1000):
    group1 = np.random.normal(loc=0.0, scale=1.0, size=30)
    group2 = np.random.normal(loc=0.0, scale=1.0, size=30)
    diff = np.mean(group2) - np.mean(group1)
    t, p = stats.ttest_ind(group1, group2,equal_var = True) 
    #t, p  = stats.wilcoxon(x = group1, y = group2) 
    resRandom.append([g,diff,t,p])

In [ ]:
fig = px.histogram(group1,nbins=10)
fig.show()

In [ ]:
# A compléter pour afficher l'histogramme de la distribution du groupe 2

2) Obtenez vous des différences significatives (p-valeur < 0.05)?

In [ ]:
resRandomTable = pd.DataFrame(resRandom, columns = ['test_number', 'diff',"T stat","p_value"])
resRandomTable[resRandomTable['p_value'] < 0.05].sort_values("p_value")

In [ ]:
# A compléter pour obtenir le nombre de différences à priori significatives à l'aide de la fonction len

3) Effectuer la correction de Benjamin et Hotchberg pour déterminer les différences significatives pour ces 1000 tests (on prendra un seuil initial sigma de 0.05).

In [ ]:
resRandomTable = resRandomTable.sort_values("p_value")
resRandomTable["corrected_threshold"]=[(r/len(resRandomTable))*0.05 for r in range(len(resRandomTable)+1)[1:]]
resRandomTable['H0 rejected'] = resRandomTable['p_value'] < resRandomTable['corrected_threshold']
resRandomTable

#### Remarque sur le code:
Nous avons utilisé ici (et nous utiliserons plus tard dans l'exercice) une liste en compréhension pour écrire de manière synthétique une boucle for: `[(r/len(resRandomTable))*0.05 for r in range(len(resRandomTable)+1)[1:]]`. Le code équivalent avec une boucle for classique, beaucoup plus long à écrire, est le suivant:

In [ ]:
corrected_threshold_list = [] #déclaration d'une liste vide
for r in range(len(resRandomTable)+1)[1:]: # range fait partir la séquence à 0 mais notre premier rang est 1
    corrected_threshold = (r/len(resRandomTable))*0.05 #calcul du nouveau seuil pour le rang donné
    corrected_threshold_list.append(corrected_threshold) #ajout du seuil à la liste
    
resRandomTable["corrected_threshold_2"] = corrected_threshold_list
resRandomTable

4) Reste-t-il des différences significatives pour ces nouveaux seuils? Si oui, dans quelle proportion par rapport au nombre de tests effectués (1000)?

In [ ]:
resRandomTable[resRandomTable['H0 rejected']]

5) Effectuer la même correction pour les tests de différences d'expression réalisés lors de la séance précédente.

Commencer par charger le tableau des résultats sauvegardés précedemment. Combien de gènes sont à priori différentiellement exprimés (on prendra p_value < 0.05) entre les echantillons métastatiques et les echantillons de tumeurs primaires?

In [ ]:
diffExpResults = pd.read_csv("../Week4/diffExpMetastaticPrimaryTumor_chr14.csv")
diffExpResults[diffExpResults["p_value"]< 0.05]

In [ ]:
# A compléter pour obtenir le nombre de différences à priori significatives à l'aide de la fonction len

Puis réaliser la correction comme précédemment.

In [ ]:
# A compléter pour réaliser la correction de Benjamin et Hotchberg

6) Combien de gènes différentiellement exprimés reste-t-il après cette correction?

In [ ]:
diffExpResults[diffExpResults['H0 rejected']].sort_values('log1p_diff')

## Partie 2: Analyse des biomarqueurs prédictifs de la survie des patients atteints de melanomes

Nous allons travailler sur le fichier d'expression génique de la cohorte de patients atteints de mélanome que nous avons normalisé en TPM (`'TCGA-SKCM.htseq_tpm.csv'`)la semaine dernière et nous aurons également besoin du fichier d'annotations des gènes (`'gencode.v22.annotation.gene.probeMap.with.length.csv'`).

Aujourd'hui, nous utiliserons en plus le fichier de suivi de la survie des patients de la cohorte (`'TCGA-SKCM.survival.tsv'`) que nous avons téléchargé à cette adresse: https://xenabrowser.net/datapages/?dataset=TCGA-SKCM.survival.tsv&host=https%3A%2F%2Fgdc.xenahubs.net&removeHub=https%3A%2F%2Fxena.treehouse.gi.ucsc.edu%3A443

Nous avons mis à votre disposition ces fichiers dans le dossier UCSC_data.

1) Charger ces fichiers dans des tableaux `smpCounts`, `geneAnnotation` et `survivalData`. Mettez en index (noms des lignes) la colonne `Ensembl_ID` pour `smpCounts`, `ìd` pour `geneAnnotations` et la colonne `sample`pour `survivalData`. On conservera la colonne d'index pour geneAnnotations et survivalData (option `drop=False`).

In [ ]:
smpCounts = pd.read_csv('../UCSC_data/TCGA-SKCM.htseq_tpm.csv')
smpCounts = smpCounts.set_index('Ensembl_ID') #his will drop Ensembl_ID column
smpCounts

In [ ]:
geneAnnotations = pd.read_csv('../UCSC_data/gencode.v22.annotation.gene.probeMap.with.length.csv')
geneAnnotations = geneAnnotations.set_index('id') #this will drop Ensembl_ID column
geneAnnotations

Dans `survivalData` la colonne `OS` correspond au dernier status vital connu du patient: 1 s'il était mort, 0 s'il etait en vie.
La colonne `OS.time` correspond au temps de survie mesuré en jours du patient jusqu'à sa mort (si `OS=1`), jusqu'à la fin de l'étude s'il était à ce moment là toujours en vie, ou jusqu'à ce qu'il quitte l'étude avant la fin de celle ci (si `OS=0`).


In [ ]:
survivalData = pd.read_table('../UCSC_data/TCGA-SKCM.survival.tsv',sep="\t")
survivalData = survivalData.set_index('sample')
survivalData

La recherche des biomarqueurs de survie dans l'ensemble des gènes séquencés étant assez longue sur un ordinateur standard, nous ne travaillerons aujourd'hui qu'avec les gènes du chromosome 11.

2) En utilisant le tableau `geneAnnotations` ne conservez dans `smpCounts`que les gènes présents sur le chromosome 11

In [ ]:
smpCounts = smpCounts.loc[geneAnnotations[geneAnnotations["chrom"] == "chr11"].index.to_list()]
smpCounts

3) Ne conserver dans `smpCounts` et `survivalData` que les patients pour lesquels nous avons à la fois le suivie de la survie et l'expression génique et ordonner les patients avec le même ordre dans les deux tableaux (colonnes de `smpCounts` et index de `survivalData`).

In [ ]:
smpCounts = smpCounts[smpCounts.columns.intersection(survivalData.index)]
survivalData = survivalData.loc[survivalData.index.intersection(smpCounts.columns)]
survivalData = survivalData.loc[smpCounts.columns]
survivalData

Nous allons utiliser le package scikit-survival pour faire l'analyse des biomarqueurs prédictifs de la surive des patients.
Celui-ci  prend les données de statuts des patients (0/1 colonnes OS de survivalData) sous forme booléenne (True / False) décés confirmés / non confirmés.

4) Ajouter une colonne `Status` à `survivalData` correspondant à la colonne `OS`sous forme booléenne.

In [ ]:
survivalData["Status"] = survivalData["OS"] == 1
survivalData

Nous allons commencer par regarder si l'expression du gène CD3G chez les patients peut être prédictif de leur survie. Ce gène code pour une sous unité du recepteur des cellules T. 

5) Séparez les patients en deux groupes selon qu'il expriment fortement ou non CD3G. On prendra la médiane d'expression de CD3G de la cohorte comme seuil pour définir ces deux groupes. Stocker le résultat dans une nouvelle colonne de survivalData.

In [ ]:
gene_name = "CD3G"
gene_id = geneAnnotations[geneAnnotations["gene"]=="CD3G"].index[0]
gene_id
survivalData[gene_id] = ['high' if i > np.median(smpCounts.loc[gene_id]) else 'low' for i in smpCounts.loc[gene_id]]
survivalData

6) Utiliser la fonction `kaplan_meier_estimator` du package scikit-survival pour tracer la courbe de survie pour chacun des deux groupes. 

In [ ]:
for gene_exp in ('low', 'high'):
    mask_exp = survivalData[gene_id] == gene_exp
    time_exp, survival_prob_exp = kaplan_meier_estimator(
        survivalData["Status"][mask_exp],
        survivalData['OS.time'][mask_exp])

    plt.step(time_exp, survival_prob_exp, where="post",
             label="Exp CD3G %s" % gene_exp)

plt.ylabel("est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$")
plt.legend(loc="best")

7) A l'aide de la fonction `compare_survival`déterminer si la différence observée semble significative.

In [ ]:
survivalData_array = survivalData[['Status','OS.time']].to_records(index=False)
compare_survival(survivalData_array, survivalData[[gene_id]].to_numpy())

8) A l'aide d'une boucle for, répétez maintenant ce test pour l'ensemble des gènes du chromosome 11. Ne tester que les gènes dont l'expression est détectée dans au moins 10 patients.

In [ ]:
res = list()
for g in smpCounts.index: 
    x = smpCounts.loc[g]
    if (np.count_nonzero(x) > 10) : 
        y = survivalData["OS.time"]
        survivalData[g] = ['high' if i > np.median(x) else 'low' for i in smpCounts.loc[g]]
        survivalData_array = survivalData[['Status','OS.time']].to_records(index=False)
        stat, p = compare_survival(survivalData_array, survivalData[[g]].to_numpy())
        res.append([g,stat,p])

In [ ]:
biomarkerResults = pd.DataFrame(res, columns = ['Ensembl_ID',"stat","p_value"])
biomarkerResults["gene_name"] = geneAnnotations["gene"].loc[biomarkerResults["Ensembl_ID"]].to_list()

9) Combien de gènes du chromosme 11 semblent à priori être prédictifs de la survie des patients (p-valeur < 0.05)? Peut-on retenir cette liste telle quelle?

In [ ]:
# A compléter

## Question bonus (à regarder chez vous, pas à l'examen): 

10) Effectuer la correction de Benjamin et Hotchberg sur ces résultats. Combien restent-t'ils de résultats significatifs?


In [ ]:
biomarkerResults = biomarkerResults.sort_values("p_value")
biomarkerResults["corrected_threshold"]=[(r/len(biomarkerResults))*0.05 for r in range(len(biomarkerResults)+1)[1:]]
biomarkerResults['H0 rejected'] = biomarkerResults['p_value'] < biomarkerResults['corrected_threshold']
biomarkerResults[biomarkerResults['H0 rejected']]

11) Est ce que les expressions des autres gènes du complexe CD3 des cellules T sont predictives de la survie des patients? Tracer les courbes de survies associées.

In [ ]:
CD3_genes = [i for i in biomarkerResults["gene_name"] if i.startswith('CD3')]

In [ ]:
biomarkerResults[biomarkerResults["gene_name"].isin(CD3_genes)]

In [ ]:
for gene_name in ["CD3G","CD3E"]:
    gene_id = geneAnnotations[geneAnnotations["gene"]==gene_name].index[0]
    survivalData[gene_id] = ['high' if i > np.median(smpCounts.loc[gene_id]) else 'low' for i in smpCounts.loc[gene_id]]
    for gene_exp in ('low', 'high'):
        mask_exp = survivalData[gene_id] == gene_exp
        time_exp, survival_prob_exp = kaplan_meier_estimator(
            survivalData["Status"][mask_exp],
            survivalData['OS.time'][mask_exp])

        plt.step(time_exp, survival_prob_exp, where="post",
             label="Exp "+gene_name +" %s" % gene_exp)

    plt.ylabel("est. probability of survival $\hat{S}(t)$")
    plt.xlabel("time $t$")
    plt.legend(loc="best")
    plt.show()

Répéter cette analyse mais cette fois ci en distinguant les différents types d'echantillons. Vous aurez besoin de charger le tableau des données cliniques: `"../UCSC_data/TCGA-SKCM.GDC_phenotype.tsv"`


In [ ]:
clinicalData = pd.read_table("../UCSC_data/TCGA-SKCM.GDC_phenotype.tsv")
clinicalData = clinicalData.set_index("submitter_id.samples",drop = False)
survivalData["sample_type"] = clinicalData.loc[survivalData.index,"sample_type.samples"]

In [ ]:
for gene_name in ["CD3D","CD3G","CD3E"]:
    gene_id = geneAnnotations[geneAnnotations["gene"]==gene_name].index[0]
    for smp_type in ('Primary Tumor', 'Metastatic'):
        survivalDataSmp = survivalData[survivalData["sample_type"] == smp_type]
        for gene_exp in ('low', 'high'):
            mask_exp = survivalDataSmp[gene_id] == gene_exp
            time_exp, survival_prob_exp = kaplan_meier_estimator(
                survivalDataSmp["Status"][mask_exp],
                survivalDataSmp['OS.time'][mask_exp])

            plt.step(time_exp, survival_prob_exp, where="post",
                     label=smp_type+" Exp "+gene_name +" %s" % gene_exp)

    plt.ylabel("est. probability of survival $\hat{S}(t)$")
    plt.xlabel("time $t$")
    plt.legend(loc="best")
    plt.show()

Afficher ces courbes de survies uniquement pour les échantillons de tumeurs primaires

In [ ]:
smp_type = "Primary Tumor"
survivalDataSmp = survivalData[survivalData["sample_type"] == smp_type]
for gene_name in ["CD3D","CD3G","CD3E"]:
    gene_id = geneAnnotations[geneAnnotations["gene"]==gene_name].index[0]
    smp_type = 'Primary Tumor'
    survivalDataSmp = survivalData[survivalData["sample_type"] == smp_type]
    for gene_exp in ('low', 'high'):
        mask_exp = survivalDataSmp[gene_id] == gene_exp
        time_exp, survival_prob_exp = kaplan_meier_estimator(
            survivalDataSmp["Status"][mask_exp],
            survivalDataSmp['OS.time'][mask_exp])

        plt.step(time_exp, survival_prob_exp, where="post",
                 label=smp_type+" Exp "+gene_name +" %s" % gene_exp)

    plt.ylabel("est. probability of survival $\hat{S}(t)$")
    plt.xlabel("time $t$")
    plt.legend(loc="best")
    plt.show()